In [5]:
import numpy as np
import pandas as pd
from tqdm import tqdm

## Medida del CRPS
def heavyside(prediction,actual):
    return prediction >= actual

def cdf_dif(prediction,actual):
    quantiles = np.arange(1,100)/100.0
    t=pd.Series(prediction)
    dif=t-t.shift(1)
    dif=dif.dropna()
    fs = sum(dif*((quantiles-heavyside(prediction,actual))[1:]**2))
    # If the actual is outside the range of the prediction, 
    # we need to account for that areas outside the range 
    if actual > prediction[-1]:
        fs += (actual-prediction[-1]) * 1
    if actual < prediction[0]:
        fs += (prediction[0]-actual) * 1
    return fs

def CRPS(predictions, actuals):
    difs_mean = [cdf_dif(predictions[i],actuals[i]) for i in range(len(actuals))]
    return np.mean(difs_mean)

def evaluate(predictions,target):

    res={}
    
    # Calculate the CRPS
    res["crps"]=CRPS(predictions,target)
    
    # Bonus useful Feature
    count = 0
    for i in range(len(target)):
        if (target[i]>predictions[i][0]) and (target[i]<predictions[i][-1]):
            count+=1
    res["count"]=count
    
    ## Calculate as well measures for the quantile 50
    total_df = pd.DataFrame(predictions)
    quantiles = np.arange(1,100)/100.0 
    total_df.columns=np.array(quantiles).astype(str)
    #RMSE       
    res["rmse"]=np.sqrt(np.mean((target-total_df["0.5"])**2))
    #MAE    
    res["mae"]=np.mean(np.abs(target-total_df["0.5"] ) )
    #Bias 
    res["bias"]=np.mean(target-total_df["0.5"])
    #Corr
    res["corr"]=np.corrcoef(target,total_df["0.5"])[0][1]

    res["all"]=predictions
    res["target"]=target
    
    return res

In [6]:
import numpy as np
import math
import pandas as pd
from tqdm import  tqdm
from sklearn.model_selection import KFold
from numpy.fft import fft
import datetime
import json
import numpy as np
from scipy import stats

def prepare_data_from_horizon(df, horizon=12):
    
    data=df.copy()

    sel = np.concatenate([[1,2,3,4],
                         [12],
                         24*np.arange(1,9),
                         12+24*np.arange(1,9)])  
    sel=np.concatenate([sel,sel-1,sel+1])  
    
    ## lagged NO2 values
    for i in sel:
        if (i>=horizon):
            data["NO2 - "+str(i)] = data["NO2"].shift(i)

    ## lagged O3 values
    for i in 24*np.arange(1,4):
        if (i>=horizon):
            data["O3 - "+str(i)] = data["O3"].shift(i)

    ## Remove empty values
    data=data.dropna()

    X=data[list(set(data.columns)-set(['DATE',"NO2","O3"]))]
    y=data["NO2"]
  
    return X, y

#"data/dataEscAgui.csv"
def get_data(path):

    df = pd.read_csv(path,sep=";")

    # Prepare data
    data = df[["DATE","SPA.NO2","SPA.O3","MACC.NO2"]].copy()
    data["DATE"]=pd.to_datetime(data["DATE"],format="%Y-%m-%d %H:%M:%S")
    data = data.sort_values("DATE")
    data.columns = ["DATE","NO2","O3","CAMS"]

    ## Remove everything from 2020
    data=data[data["DATE"].astype(str)<"2020"]

    ## Fourier Columns
    freqs = [2922,1461,209,1465,4]
    l = 35064
    n = np.arange(len(data))
    fcols = []
    for f in freqs:
        data["c"+str(f)]=np.cos(n*2*np.pi*f/l)
        fcols.append("c"+str(f))
        data["s"+str(f)]=np.cos(n*2*np.pi*f/l)
        fcols.append("s"+str(f))

    data["NO2"]=np.log1p(data["NO2"])
    data["O3"]=np.log1p(data["O3"])
    data["CAMS"]=np.log1p(data["CAMS"])

    ## Calendar Variables 
    ## Calendar Variables do not bring better results and therefore
    ## removed
    
    return data

In [37]:
import numpy as np
import math
import pandas as pd
from tqdm import tqdm
from scipy import stats

from keras.models import Sequential
from keras.layers import Dense, LeakyReLU
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
import keras.backend as K 

from sklearn.model_selection import train_test_split

def tilted_loss(q,y,f):
    e = (y-f)
    return K.mean(K.maximum(q*e, (q-1)*e), axis=-1)

class MLPQuantile():
    
    def __init__(self,batch_size,epochs,lr):
        self.batch_size = batch_size
        self.epochs = epochs
        self.lr=lr
        self.estimators = []
        
    def fit(self,X_train,y_train):
        
        def MLPmodel():
            model = Sequential()
            model.add(Dense(len(X_train[0]), input_dim=len(X_train[0]), activation=LeakyReLU(alpha=0.3)))
            model.add(Dense(int(len(X_train[0])/2), activation=LeakyReLU(alpha=0.3)))
            model.add(Dense(1, activation='linear'))
            return model
        
        print("training !")

        X_ttrain, X_val, y_ttrain, y_val = train_test_split(X_train,y_train,test_size=.05,random_state=2020)

        for q in [0.022750131948179195,0.15865525393145707,0.5,0.8413447460685429,0.9772498680518208]:
            print(f"Quantile: {q}")
            model = MLPmodel()
            optim=Adam(learning_rate=self.lr)
            model.compile(loss=lambda y,f: tilted_loss(q,y,f), optimizer=optim)
            es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=10)
            history = model.fit(X_ttrain, y_ttrain, 
                                epochs=self.epochs, batch_size=self.batch_size,  
                                verbose=1,callbacks=[es],
                                validation_data=(X_val,y_val))
            self.estimators.append(model)
        print("Done")
        
    def predict(self,X):
        predictions_gbr = []
        print("predicting")
        for reg in tqdm(self.estimators):
            predictions_gbr.append(reg.predict(X))
         
        total_pred={}
        for i in range(len(predictions_gbr)):
            total_pred[i]=predictions_gbr[i][:,0]
            
        total_df=pd.DataFrame(total_pred)

        def process_row(row):
            v = row.values
            dif_mean = np.abs(v-v[2])
            mu = v[2]
            s = np.mean([dif_mean[0]/2,dif_mean[1],dif_mean[3],dif_mean[4]/2])
            mi_norm = stats.norm(mu,s)
            quant=[]
            for quantile in np.arange(1,100)/100.0 :
                quant.append(mi_norm.ppf(quantile))
            return pd.Series(quant)
 
        total_df = total_df.apply(process_row,axis=1)
        
        return total_df.values


In [38]:
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold

In [39]:
horizon = 13

df = get_data("../2018_2019_data/28079008.csv")
X, y = prepare_data_from_horizon(df,horizon)

kf = KFold(5,shuffle=True)
train_index, test_index = list(kf.split(X))[1]

train_index = X.index.values[train_index]
test_index = X.index.values[test_index] 
        
# Filter the test index when prediction time is 10:00
ten_index = df[(df["DATE"]-timedelta(hours=horizon)).dt.hour==10].index
test_index_10 = test_index[pd.Series(test_index).isin(ten_index)]
        
# We retrieve the indexes that are related to the test indexes according to our AR model
sel = np.concatenate([[1,2,3,4],
                            [12],
                            24*np.arange(1,9),
                            12+24*np.arange(1,9)])  
sel=np.concatenate([sel,sel-1,sel+1]) 
        
all_index_related_test = set([])
for i in sel:
    all_index_related_test |= set(test_index_10+i)
        
train_index_CV = train_index[pd.Series(train_index).isin(list(all_index_related_test))]
        
X_train = X.loc[train_index_CV]
y_train = y.loc[train_index_CV]
        
X_test = X.loc[test_index_10]
y_test = y.loc[test_index_10]
        
scaler = RobustScaler()
# Fit the scaler on the training features and transform these in one go
X_train_std = scaler.fit_transform(X_train)
# Scale the test set
X_test_std = scaler.transform(X_test)
        
lin = LinearRegression()
lin.fit(X_train_std,y_train)

dif_train = y_train-lin.predict(X_train_std)
dif_test = y_test-lin.predict(X_test_std)

In [40]:
from tqdm.notebook import tqdm

In [41]:
alls=[]

for batch_size in tqdm([32,64,200,500,1000,2000]):
    for epochs in tqdm([200]):
        for lr in tqdm([0.0001,0.001,0.01,0.1]):

            qreg = MLPQuantile(batch_size=batch_size,epochs=epochs,lr=lr)
            qreg.fit(X_train_std,y_train)
            preds = qreg.predict(X_test_std)
            results=evaluate((np.exp(preds)-1),(np.exp(y_test)-1).values)

            del results["all"]
            del results["target"]
            results["batch_size"]=batch_size
            results["epochs"]=epochs
            results["lr"]=lr
            alls.append(results)

training !
Quantile: 0.022750131948179195
Epoch 1/200
327/327 [==============================] - 1s 1ms/step - loss: 0.0754 - val_loss: 0.0528
Epoch 2/200
327/327 [==============================] - 0s 864us/step - loss: 0.0524 - val_loss: 0.0412
Epoch 3/200
327/327 [==============================] - 0s 923us/step - loss: 0.0413 - val_loss: 0.0351
Epoch 4/200
327/327 [==============================] - 0s 954us/step - loss: 0.0371 - val_loss: 0.0315
Epoch 5/200
327/327 [==============================] - 0s 919us/step - loss: 0.0336 - val_loss: 0.0304
Epoch 6/200
327/327 [==============================] - 0s 974us/step - loss: 0.0325 - val_loss: 0.0294
Epoch 7/200
327/327 [==============================] - 0s 943us/step - loss: 0.0308 - val_loss: 0.0285
Epoch 8/200
327/327 [==============================] - 0s 870us/step - loss: 0.0294 - val_loss: 0.0280
Epoch 9/200
327/327 [==============================] - 0s 919us/step - loss: 0.0282 - val_loss: 0.0272
Epoch 10/200
327/327 [===========

327/327 [==============================] - 0s 841us/step - loss: 0.6620 - val_loss: 0.3936
Epoch 3/200
327/327 [==============================] - 0s 816us/step - loss: 0.3772 - val_loss: 0.3072
Epoch 4/200
327/327 [==============================] - 0s 846us/step - loss: 0.3021 - val_loss: 0.2730
Epoch 5/200
327/327 [==============================] - 0s 839us/step - loss: 0.2678 - val_loss: 0.2532
Epoch 6/200
327/327 [==============================] - 0s 839us/step - loss: 0.2440 - val_loss: 0.2394
Epoch 7/200
327/327 [==============================] - 0s 834us/step - loss: 0.2251 - val_loss: 0.2257
Epoch 8/200
327/327 [==============================] - 0s 821us/step - loss: 0.2132 - val_loss: 0.2163
Epoch 9/200
327/327 [==============================] - 0s 836us/step - loss: 0.2038 - val_loss: 0.2069
Epoch 10/200
327/327 [==============================] - 0s 832us/step - loss: 0.1964 - val_loss: 0.1948
Epoch 11/200
327/327 [==============================] - 0s 877us/step - loss: 0.1875

Epoch 81/200
327/327 [==============================] - 0s 831us/step - loss: 0.1193 - val_loss: 0.1367
Epoch 82/200
327/327 [==============================] - 0s 824us/step - loss: 0.1170 - val_loss: 0.1362
Epoch 83/200
327/327 [==============================] - 0s 826us/step - loss: 0.1174 - val_loss: 0.1357
Epoch 84/200
327/327 [==============================] - 0s 833us/step - loss: 0.1183 - val_loss: 0.1375
Epoch 85/200
327/327 [==============================] - 0s 846us/step - loss: 0.1173 - val_loss: 0.1371
Epoch 86/200
327/327 [==============================] - 0s 831us/step - loss: 0.1180 - val_loss: 0.1377
Epoch 87/200
327/327 [==============================] - 0s 826us/step - loss: 0.1162 - val_loss: 0.1379
Epoch 88/200
327/327 [==============================] - 0s 825us/step - loss: 0.1167 - val_loss: 0.1366
Epoch 89/200
327/327 [==============================] - 0s 826us/step - loss: 0.1173 - val_loss: 0.1363
Epoch 90/200
327/327 [==============================] - 0s 976us

327/327 [==============================] - 0s 847us/step - loss: 0.0199 - val_loss: 0.0244
Epoch 45/200
327/327 [==============================] - 0s 853us/step - loss: 0.0199 - val_loss: 0.0245
Epoch 46/200
327/327 [==============================] - 0s 822us/step - loss: 0.0196 - val_loss: 0.0248
Epoch 47/200
327/327 [==============================] - 0s 824us/step - loss: 0.0191 - val_loss: 0.0238
Epoch 48/200
327/327 [==============================] - 0s 814us/step - loss: 0.0193 - val_loss: 0.0242
Epoch 49/200
327/327 [==============================] - 0s 821us/step - loss: 0.0190 - val_loss: 0.0247
Epoch 50/200
327/327 [==============================] - 0s 844us/step - loss: 0.0191 - val_loss: 0.0238
Epoch 51/200
327/327 [==============================] - 0s 816us/step - loss: 0.0188 - val_loss: 0.0238
Epoch 52/200
327/327 [==============================] - 0s 822us/step - loss: 0.0184 - val_loss: 0.0239
Epoch 53/200
327/327 [==============================] - 0s 844us/step - loss:


training !
Quantile: 0.022750131948179195
Epoch 1/200
327/327 [==============================] - 1s 1ms/step - loss: 0.0544 - val_loss: 0.0290
Epoch 2/200
327/327 [==============================] - 0s 845us/step - loss: 0.0296 - val_loss: 0.0274
Epoch 3/200
327/327 [==============================] - 0s 841us/step - loss: 0.0269 - val_loss: 0.0231
Epoch 4/200
327/327 [==============================] - 0s 845us/step - loss: 0.0236 - val_loss: 0.0244
Epoch 5/200
327/327 [==============================] - 0s 827us/step - loss: 0.0232 - val_loss: 0.0227
Epoch 6/200
327/327 [==============================] - 0s 845us/step - loss: 0.0216 - val_loss: 0.0232
Epoch 7/200
327/327 [==============================] - 0s 825us/step - loss: 0.0211 - val_loss: 0.0226
Epoch 8/200
327/327 [==============================] - 0s 817us/step - loss: 0.0212 - val_loss: 0.0208
Epoch 9/200
327/327 [==============================] - 0s 818us/step - loss: 0.0209 - val_loss: 0.0223
Epoch 10/200
327/327 [==========

...

Epoch 20/200
11/11 [==============================] - 0s 4ms/step - loss: 0.0222 - val_loss: 0.0254
Epoch 21/200
11/11 [==============================] - 0s 4ms/step - loss: 0.0227 - val_loss: 0.0243
Epoch 22/200
11/11 [==============================] - 0s 4ms/step - loss: 0.0224 - val_loss: 0.0241
Epoch 23/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0245 - val_loss: 0.0241
Epoch 24/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0219 - val_loss: 0.0246
Epoch 25/200
11/11 [==============================] - 0s 4ms/step - loss: 0.0209 - val_loss: 0.0233
Epoch 26/200
11/11 [==============================] - 0s 4ms/step - loss: 0.0214 - val_loss: 0.0253
Epoch 27/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0223 - val_loss: 0.0224
Epoch 28/200
11/11 [==============================] - 0s 4ms/step - loss: 0.0205 - val_loss: 0.0240
Epoch 29/200
11/11 [==============================] - 0s 4ms/step - loss: 0.0195 - val_loss: 0.0216



training !
Quantile: 0.022750131948179195
Epoch 1/200
11/11 [==============================] - 0s 16ms/step - loss: 3.3935 - val_loss: 0.5360
Epoch 2/200
11/11 [==============================] - 0s 5ms/step - loss: 0.5642 - val_loss: 0.3524
Epoch 3/200
11/11 [==============================] - 0s 5ms/step - loss: 0.4198 - val_loss: 0.6729
Epoch 4/200
11/11 [==============================] - 0s 5ms/step - loss: 0.5060 - val_loss: 0.2218
Epoch 5/200
11/11 [==============================] - 0s 6ms/step - loss: 0.1762 - val_loss: 0.0997
Epoch 6/200
11/11 [==============================] - 0s 5ms/step - loss: 0.1071 - val_loss: 0.2910
Epoch 7/200
11/11 [==============================] - 0s 5ms/step - loss: 0.2656 - val_loss: 0.1632
Epoch 8/200
11/11 [==============================] - 0s 5ms/step - loss: 0.1309 - val_loss: 0.0934
Epoch 9/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0781 - val_loss: 0.0718
Epoch 10/200
11/11 [==============================] - 0s 5ms/step

11/11 [==============================] - 0s 5ms/step - loss: 0.0859 - val_loss: 0.0906
Epoch 22/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0833 - val_loss: 0.0915
Epoch 23/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0824 - val_loss: 0.0905
Epoch 24/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0806 - val_loss: 0.0864
Epoch 25/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0796 - val_loss: 0.0867
Epoch 26/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0787 - val_loss: 0.0945
Epoch 27/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0808 - val_loss: 0.0879
Epoch 28/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0791 - val_loss: 0.0909
Epoch 29/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0828 - val_loss: 0.0851
Epoch 30/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0816 - val_loss: 0.0943
Epoch 31/200


<ipython-input-41-d85b4a661713>:11: RuntimeWarning: overflow encountered in exp
  results=evaluate((np.exp(preds)-1),(np.exp(y_test)-1).values)


ValueError: operands could not be broadcast together with shapes (97,) (98,) 

In [42]:
pd.DataFrame(alls).sort_values("rmse")

,crps,count,rmse,mae,bias,corr,batch_size,epochs,lr
1,1.321016e+01,448,26.616627,18.391448,0.051723,0.721878,32,200,0.0010
5,1.365685e+01,437,27.098976,19.142774,-0.828149,0.713024,64,200,0.0010
14,1.373145e+01,452,27.349485,19.145823,0.471469,0.705730,500,200,0.0100
8,1.387997e+01,449,27.351222,19.479169,0.566238,0.703048,200,200,0.0001
0,1.370337e+01,447,27.488187,19.324941,0.986346,0.700785,32,200,0.0001
17,1.428967e+01,442,27.954319,19.786370,-2.448371,0.698197,1000,200,0.0010
6,1.420735e+01,451,28.168830,19.483845,2.873022,0.693409,64,200,0.0100
18,1.355074e+01,453,28.185922,19.271307,7.253688,0.709076,1000,200,0.0100
2,1.402934e+01,449,28.254392,19.538328,2.440787,0.685368,32,200,0.0100
4,1.449076e+01,447,28.820583,20.832350,1.122662,0.664688,64,200,0.0001
